In [1]:
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')
from functools import reduce
import re
import numpy as np
import datetime
from datetime import date, timedelta

In [2]:
def Percentage(x,y):
    if x == 0 or y == 0:
        return 0
    else :
        quotient = x / y
        percent = quotient * 100
        percent = int(percent)
        return percent

In [3]:
DemandTracker=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\DemandTracker.xlsx')
OnboardingTracker=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\OnboardingTracker.xlsx')
HCL=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\HCL.xlsx')
INFY=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\Infy.xlsx')

In [4]:
TribeName = 'BO'

In [5]:
TribeDemandTracker = DemandTracker[DemandTracker['Tribe']=='BUSINESS OWNERS']
TribeOnboardingTracker = OnboardingTracker[OnboardingTracker['Tribe']=='BUSINESS OWNERS']
TribeHCL = HCL[HCL['Tribe']=='BUSINESS OWNERS']
TribeINFY = INFY[INFY['Tribe']=='BUSINESS OWNERS']

Success Rate

In [6]:
tribe = TribeDemandTracker

In [7]:
tribe = tribe.rename(columns={'Demand Status':'Demand_Status'})

In [8]:
role = DemandTracker['Role'].unique()
role= pd.DataFrame(role)
role.columns=['Role']

In [9]:
role

,Role
0,Analyst
1,Engineer
2,Tester
3,Test Lead
4,Environment Manager
5,Release Lead
6,Tech Area Architect
7,Operations Expert
8,Agile Coach
9,Solution Designer


In [10]:
cls = tribe[tribe['Demand_Status']=='Closed']
cls=cls.groupby(['Role'])['Demand ID'].count().reset_index(name='Closed')

In [11]:
opn = tribe[tribe['Demand_Status']=='Open']
opn=opn.groupby(['Role'])['ID'].count().reset_index(name='Open')

In [12]:
opn

,Role,Open


In [13]:
wdr = tribe[tribe['Demand_Status']=='Withdrawn']
wdr=wdr.groupby(['Role'])['ID'].count().reset_index(name='Withdrawn')

In [14]:
opnbkf = tribe[tribe['Demand_Status']=='Open Backfill']
opnbkf=opnbkf.groupby(['Role'])['ID'].count().reset_index(name='Open_Backfill')

In [15]:
clsbkf = tribe[tribe['Demand_Status']=='Backfill Closed']
clsbkf=clsbkf.groupby(['Role'])['ID'].count().reset_index(name='Backfill_Closed')

In [16]:
demandttl=[cls,opn,wdr,opnbkf,clsbkf,role]

In [17]:
TotalDemand=reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),demandttl)

In [18]:
TotalDemand[['Closed','Open','Withdrawn','Open_Backfill','Backfill_Closed']]=TotalDemand[['Closed','Open','Withdrawn','Open_Backfill','Backfill_Closed']].fillna(0)
TotalDemand[['Closed','Open','Withdrawn','Open_Backfill','Backfill_Closed']]=TotalDemand[['Closed','Open','Withdrawn','Open_Backfill','Backfill_Closed']].astype(int)

In [19]:
TotalDemand

,Role,Closed,Open,Withdrawn,Open_Backfill,Backfill_Closed
0,Engineer,1,0,0,0,0
1,Solution Designer,2,0,0,0,0
2,Analyst,0,0,0,0,0
3,Tester,0,0,0,0,0
4,Test Lead,0,0,0,0,0
5,Environment Manager,0,0,0,0,0
6,Release Lead,0,0,0,0,0
7,Tech Area Architect,0,0,0,0,0
8,Operations Expert,0,0,0,0,0
9,Agile Coach,0,0,0,0,0


In [20]:
TotalDemand['Grand Total'] = TotalDemand['Backfill_Closed'] + TotalDemand['Closed'] + TotalDemand['Open']

In [21]:
TotalDemand.loc['Total'] = pd.Series(TotalDemand['Grand Total'].sum(), index = ['Grand Total'])

In [22]:
TotalDemand = TotalDemand.replace(np.nan, '', regex=True)

In [23]:
TotalDemand['Grand Total'] = TotalDemand['Grand Total'].astype(int)

In [24]:
TotalDemand

,Role,Closed,Open,Withdrawn,Open_Backfill,Backfill_Closed,Grand Total
0,Engineer,1,0,0,0,0,1
1,Solution Designer,2,0,0,0,0,2
2,Analyst,0,0,0,0,0,0
3,Tester,0,0,0,0,0,0
4,Test Lead,0,0,0,0,0,0
5,Environment Manager,0,0,0,0,0,0
6,Release Lead,0,0,0,0,0,0
7,Tech Area Architect,0,0,0,0,0,0
8,Operations Expert,0,0,0,0,0,0
9,Agile Coach,0,0,0,0,0,0


In [25]:
TotalDemand.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Demand_'+TribeName+'.csv',sep=',',index=None)

In [26]:
TotalDemand

,Role,Closed,Open,Withdrawn,Open_Backfill,Backfill_Closed,Grand Total
0,Engineer,1,0,0,0,0,1
1,Solution Designer,2,0,0,0,0,2
2,Analyst,0,0,0,0,0,0
3,Tester,0,0,0,0,0,0
4,Test Lead,0,0,0,0,0,0
5,Environment Manager,0,0,0,0,0,0
6,Release Lead,0,0,0,0,0,0
7,Tech Area Architect,0,0,0,0,0,0
8,Operations Expert,0,0,0,0,0,0
9,Agile Coach,0,0,0,0,0,0


Onboarding Status by Role

In [27]:
DXCOnboarding = TribeOnboardingTracker
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
DXCOnboarding.loc[DXCOnboarding['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'

In [28]:
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Test Lead'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Environment Manager'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Operations Expert'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Agile Coach'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Solution Designer'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Developer'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Transition Manager'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Product Owner'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Lead Developer'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Journey Expert'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['SDET - Automation Tester'],['Tester'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Chapter Lead'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['PMO Analyst'],['Other'])
DXCOnboarding['Role'] = DXCOnboarding['Role'].replace(['Delivery Partner'],['Other'])

In [29]:
tribeOnb = DXCOnboarding[((DXCOnboarding['Onboarding Status']=='Onboarding Completed'))]

In [30]:
OnboardingCompleteResource = tribeOnb[['Partner','Resource Name','Role','Technical Start Date','Squad']]

In [31]:
OnboardingCompleteResource = OnboardingCompleteResource.fillna('')

In [32]:
OnboardingCompleteResource.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Onboarding_Complete_'+TribeName+'.csv',sep=',',index=None)

In [33]:
tribeOnb=tribeOnb.groupby(['Role'])['ID'].count().reset_index(name='Onboarding Completed')

In [34]:
tribeInprg=DXCOnboarding[((DXCOnboarding['Onboarding Status']=='Onboarding In Progress'))]
OnboardingInProgressResource = tribeInprg[['Partner','Resource Name','Role','Technical Start Date','Squad',]]

In [35]:
OnboardingInProgressResource = OnboardingInProgressResource.fillna('')

In [36]:
#OnboardingInProgressResource['Technical Start Date'] = pd.to_datetime(OnboardingInProgressResource['Technical Start Date'],format='%Y-%m-%d')
#OnboardingInProgressResource['Technical Start Date'] = OnboardingInProgressResource['Technical Start Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [37]:
OnboardingInProgressResource.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Onboarding_InProgress_'+TribeName+'.csv',sep=',',index=None)

In [38]:
tribeInprg=tribeInprg.groupby(['Role'])['ID'].count().reset_index(name='Onboarding In Progress')

In [39]:
tribeinit=DXCOnboarding[((DXCOnboarding['Onboarding Status']=='Onboarding Initiated'))]
OnboardingInitiated = tribeinit[['Partner','Resource Name','Role','Technical Start Date','Squad']]
OnboardingInitiated = OnboardingInitiated.fillna('')
#OnboardingInitiated['Technical Start Date'] = pd.to_datetime(OnboardingInitiated['Technical Start Date'],format='%Y-%m-%d')
#OnboardingInitiated['Technical Start Date'] = OnboardingInitiated['Technical Start Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [40]:
OnboardingInitiated.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Onboarding_Initiated_'+TribeName+'.csv',sep=',',index=None)

In [41]:
tribeinit=tribeinit.groupby(['Role'])['ID'].count().reset_index(name='Onboarding Initiated')

In [42]:
DefaultRoles=['Engineer','Tester','Release Lead','Analyst','Tech Area Architect','Scrum Master','Other']
DefaultRoles = pd.DataFrame(DefaultRoles)
DefaultRoles.columns=['Role']

In [43]:
ttlonb=[tribeOnb,tribeInprg,tribeinit,DefaultRoles]

In [44]:
TotalOnboarded=reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),ttlonb)

In [45]:
TotalOnboarded=TotalOnboarded[['Role','Onboarding Completed','Onboarding In Progress','Onboarding Initiated']]

In [46]:
TotalOnboarded[['Onboarding Completed','Onboarding In Progress','Onboarding Initiated']]=TotalOnboarded[['Onboarding Completed','Onboarding In Progress','Onboarding Initiated']].fillna(0)
TotalOnboarded[['Onboarding Completed','Onboarding In Progress','Onboarding Initiated']]=TotalOnboarded[['Onboarding Completed','Onboarding In Progress','Onboarding Initiated']].astype(int)

In [47]:
TotalOnboarded = TotalOnboarded.transpose()
TotalOnboarded.columns = TotalOnboarded.iloc[0]
TotalOnboarded = TotalOnboarded[1:]
TotalOnboarded = TotalOnboarded[['Engineer','Tester','Release Lead','Analyst','Tech Area Architect','Scrum Master','Other']]

In [48]:
TotalOnboarded

Role,Engineer,Tester,Release Lead,Analyst,Tech Area Architect,Scrum Master,Other
Onboarding Completed,0,0,0,0,0,0,0
Onboarding In Progress,1,0,0,0,0,0,1
Onboarding Initiated,0,0,0,0,0,0,0


In [49]:
TotalOnboarded['Total']=TotalOnboarded.sum(axis=1) 
TotalOnboarded['Total'] = TotalOnboarded['Total'].astype(int)

In [50]:
TotalOnboarded

Role,Engineer,Tester,Release Lead,Analyst,Tech Area Architect,Scrum Master,Other,Total
Onboarding Completed,0,0,0,0,0,0,0,0
Onboarding In Progress,1,0,0,0,0,0,1,2
Onboarding Initiated,0,0,0,0,0,0,0,0


In [51]:
TotalOnboarded.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Total_Onboarded_'+TribeName+'.csv',sep=',')

In [52]:
HCLRoles = TribeHCL
HCLRoles.loc[HCLRoles['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
#HCLRoles.loc[HCLRoles['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
HCLRoles.loc[HCLRoles['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
HCLRoles.loc[HCLRoles['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'
HCLHired = HCLRoles[HCLRoles['Application Status']=='Hired']
HCLHired = HCLHired.groupby(['Role'])['ID'].count().reset_index(name='HCLHire')
HCLRoles=HCLRoles.groupby(['Role'])['ID'].count().reset_index(name='HCL')

In [53]:
InfyRoles = TribeINFY
InfyRoles.loc[InfyRoles['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
#InfyRoles.loc[InfyRoles['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
InfyRoles.loc[InfyRoles['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
InfyRoles.loc[InfyRoles['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'
InfyHired = InfyRoles[InfyRoles['Application Status']=='Hired']
InfyHired = InfyHired.groupby(['Role'])['ID'].count().reset_index(name='InfyHire')
InfyRoles=InfyRoles.groupby(['Role'])['ID'].count().reset_index(name='Infosys')

In [54]:
ps = [HCLHired,InfyHired,HCLRoles,InfyRoles,role]

In [55]:
Profiles=reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),ps)

In [56]:
Profiles

,Role,HCLHire,InfyHire,HCL,Infosys
0,Engineer,1.0,NaN,1.0,NaN
1,Solution Designer,NaN,NaN,1.0,NaN
2,Analyst,NaN,NaN,NaN,NaN
3,Tester,NaN,NaN,NaN,NaN
4,Test Lead,NaN,NaN,NaN,NaN
5,Environment Manager,NaN,NaN,NaN,NaN
6,Release Lead,NaN,NaN,NaN,NaN
7,Tech Area Architect,NaN,NaN,NaN,NaN
8,Operations Expert,NaN,NaN,NaN,NaN
9,Agile Coach,NaN,NaN,NaN,NaN


In [57]:
Profiles[['HCLHire','InfyHire','HCL','Infosys']]=Profiles[['HCLHire','InfyHire','HCL','Infosys']].fillna(0)
Profiles[['HCLHire','InfyHire','HCL','Infosys']]=Profiles[['HCLHire','InfyHire','HCL','Infosys']].astype(int)

In [58]:
Profiles['ProfilesSupplied']= Profiles['HCL'] +  Profiles['Infosys']

In [59]:
Profiles['Hire'] = Profiles['HCLHire']+Profiles['InfyHire']

In [60]:
ProfileSuppliedRoleVendor = Profiles[['Role','HCL','Infosys','ProfilesSupplied']]

In [61]:
ProfileSuppliedRoleVendor = ProfileSuppliedRoleVendor.append(ProfileSuppliedRoleVendor.sum(numeric_only=True), ignore_index=True)

In [62]:
ProfileSuppliedRoleVendor[['HCL','Infosys','ProfilesSupplied']]=ProfileSuppliedRoleVendor[['HCL','Infosys','ProfilesSupplied']].astype(int)

In [63]:
ProfileSuppliedRoleVendor = ProfileSuppliedRoleVendor.fillna('Total')

In [64]:
ProfileSuppliedRoleVendor

,Role,HCL,Infosys,ProfilesSupplied
0,Engineer,1,0,1
1,Solution Designer,1,0,1
2,Analyst,0,0,0
3,Tester,0,0,0
4,Test Lead,0,0,0
5,Environment Manager,0,0,0
6,Release Lead,0,0,0
7,Tech Area Architect,0,0,0
8,Operations Expert,0,0,0
9,Agile Coach,0,0,0


In [65]:
success = Profiles[['Role','Hire',]]

In [66]:
successdemand = TotalDemand[['Role','Grand Total']]

In [67]:
sr = [success,successdemand]

In [68]:
Successrate=reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),sr)

In [69]:
Successrate = Successrate.dropna()

In [70]:
Successrate=Successrate.rename(columns={'Grand Total':'Total Demand','Hire':'Total Hired'})
Successrate = Successrate[['Role','Total Demand','Total Hired']]

In [71]:
Successrate

,Role,Total Demand,Total Hired
0,Engineer,1,1.0
1,Solution Designer,2,0.0
2,Analyst,0,0.0
3,Tester,0,0.0
4,Test Lead,0,0.0
5,Environment Manager,0,0.0
6,Release Lead,0,0.0
7,Tech Area Architect,0,0.0
8,Operations Expert,0,0.0
9,Agile Coach,0,0.0


In [72]:
Successrate = Successrate.append(Successrate.sum(numeric_only=True), ignore_index=True)

In [73]:
Successrate= Successrate.fillna('Total')

In [74]:
Successrate[['Total Demand','Total Hired']]=Successrate[['Total Demand','Total Hired']].astype(int)

In [75]:
Successrate

,Role,Total Demand,Total Hired
0,Engineer,1,1
1,Solution Designer,2,0
2,Analyst,0,0
3,Tester,0,0
4,Test Lead,0,0
5,Environment Manager,0,0
6,Release Lead,0,0
7,Tech Area Architect,0,0
8,Operations Expert,0,0
9,Agile Coach,0,0


In [76]:
Successrate['Success Rate']=Successrate.apply(lambda x: Percentage(x['Total Hired'],x['Total Demand']), axis=1)

In [77]:
Successrate.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Successrate_'+TribeName+'.csv',sep=',',index=None)
Successrate

,Role,Total Demand,Total Hired,Success Rate
0,Engineer,1,1,100
1,Solution Designer,2,0,0
2,Analyst,0,0,0
3,Tester,0,0,0
4,Test Lead,0,0,0
5,Environment Manager,0,0,0
6,Release Lead,0,0,0
7,Tech Area Architect,0,0,0
8,Operations Expert,0,0,0
9,Agile Coach,0,0,0


In [78]:
ProfileSuppliedRoleVendor.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\ProfileSuppliedRoleVendor_'+TribeName+'.csv',sep=',',index=None)

In [79]:
rsgnshwOnboarding = TribeOnboardingTracker
rsgnshwOnboarding.loc[rsgnshwOnboarding['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
#rsgnshwOnboarding.loc[rsgnshwOnboarding['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
rsgnshwOnboarding.loc[rsgnshwOnboarding['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
rsgnshwOnboarding.loc[rsgnshwOnboarding['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'

In [80]:
tribeResign = rsgnshwOnboarding[((rsgnshwOnboarding['Onboarding Status']=='Resigned'))]
tribeResign=tribeResign.groupby(['Role'])['ID'].count().reset_index(name='Resigned')

In [81]:
demand = TotalDemand[['Role','Grand Total']]

In [82]:
demand

,Role,Grand Total
0,Engineer,1
1,Solution Designer,2
2,Analyst,0
3,Tester,0
4,Test Lead,0
5,Environment Manager,0
6,Release Lead,0
7,Tech Area Architect,0
8,Operations Expert,0
9,Agile Coach,0


In [83]:
rsg = [tribeResign,demand]

In [84]:
Resigned=reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),rsg)

In [85]:
Resigned = Resigned.dropna()

In [86]:
Resigned=Resigned.rename(columns={'Grand Total':'Demand'})

In [87]:
Resigned = Resigned[['Role','Demand','Resigned']]

In [88]:
if Resigned.empty:
    Resigned.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Resign_'+TribeName+'.csv',sep=',',index=None)
else:
    Resigned = Resigned.append(Resigned.sum(numeric_only=True), ignore_index=True)
    Resigned[['Demand','Resigned']]=Resigned[['Demand','Resigned']].astype(int)
    Resigned['Percentage Resigned'] = Resigned.apply(lambda x: Percentage(x['Resigned'],x['Demand']), axis=1)
    Resigned = Resigned.fillna('Total')
    Resigned.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Resign_'+TribeName+'.csv',sep=',',index=None)
    

In [89]:
nshwOnboarding = TribeOnboardingTracker
nshwOnboarding.loc[nshwOnboarding['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
#rsgnshwOnboarding.loc[rsgnshwOnboarding['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
nshwOnboarding.loc[nshwOnboarding['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
nshwOnboarding.loc[nshwOnboarding['Role'].str.contains('SDET - Automation Tester'),'Role'] = 'Tester'

In [90]:
tribeNshow = nshwOnboarding[((nshwOnboarding['Onboarding Status']=='No Show'))]

In [91]:
tribeNshow

,ID,Title,Position#,Demand ID,R-Type,Partner,DRO Source,Resource Name,Role,Exp. Level,...,Device Type,Workstation ID,Device Serial #,Edge Compliance,Work Order#,Comments,Onboarding Req Date,Modified By,Item Type,Path


In [92]:
tribeNshow=tribeNshow.groupby(['Role'])['ID'].count().reset_index(name='DropOff')

In [93]:
tribeNshow

,Role,DropOff


In [94]:
nshw = [tribeNshow,demand]

In [95]:
NoShow=reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),nshw)

In [96]:
NoShow = NoShow.dropna()

In [97]:
NoShow=NoShow.rename(columns={'Grand Total':'Demand'})

In [98]:
NoShow= NoShow[['Role','Demand','DropOff']]

In [99]:
NoShow

,Role,Demand,DropOff


In [100]:
if NoShow.empty:
    NoShow.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\NoShow_'+TribeName+'.csv',sep=',',index=None)
else:
    NoShow = NoShow.append(NoShow.sum(numeric_only=True), ignore_index=True)
    NoShow[['Demand','DropOff']]=NoShow[['Demand','DropOff']].astype(int)
    NoShow['Percentage Dropoffs'] = NoShow.apply(lambda x: Percentage(x['DropOff'],x['Demand']), axis=1)
    NoShow = NoShow.fillna('Total')
    NoShow.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\NoShow_'+TribeName+'.csv',sep=',',index=None)    

In [101]:
onshorePublished = TribeDemandTracker[((TribeDemandTracker['Location']=='ANZ - Melbourne'))]

In [102]:
onshorePublished=onshorePublished['ID'].count()

In [103]:
onshorePublished

2

In [104]:
onshorePublishedClosed = TribeDemandTracker[((TribeDemandTracker['Location']=='ANZ - Melbourne') & (TribeDemandTracker['Demand Status']=='Closed'))]

In [105]:
onshorePublishedClosed=onshorePublishedClosed['ID'].count()

In [106]:
onshorePublishedClosed

2

In [107]:
OnshoreResign = DXCOnboarding[((DXCOnboarding['Location']=='ANZ - Melbourne') & (DXCOnboarding['Onboarding Status']=='Resigned'))]

In [108]:
resourceResigned = OnshoreResign[['Resource Name','Role','Onboarding Status']]

In [109]:
resourceResigned

,Resource Name,Role,Onboarding Status


In [110]:
OnshoreResign=OnshoreResign['ID'].count()

In [111]:
OnshoreResign

0

In [112]:
OnshoreNshow = DXCOnboarding[((DXCOnboarding['Location']=='ANZ - Melbourne') & (DXCOnboarding['Onboarding Status']=='No Show'))]

In [113]:
resourceNoshow = OnshoreNshow[['Resource Name','Role','Onboarding Status']]

In [114]:
resourceNoshow

,Resource Name,Role,Onboarding Status


In [115]:
OnshoreNshow=OnshoreNshow['ID'].count()

In [116]:
OnshoreNshow

0

In [117]:
Onsite = [onshorePublished,onshorePublishedClosed,OnshoreResign,OnshoreNshow]

In [118]:
OnsiteUpdate=pd.DataFrame(Onsite)

In [119]:
OnsiteUpdate

,0
0,2
1,2
2,0
3,0


In [120]:
OnsiteUpdate.columns=['Counts']

In [121]:
OnsitePublished = ['TotalOnshoreDemand','Onboarded Onshore','Resigned','No Show']

In [122]:
OnsiteUpdate['Attributes']=OnsitePublished

In [123]:
OnsiteUpdate = OnsiteUpdate[['Attributes','Counts']]

In [124]:
OnsiteUpdate = OnsiteUpdate.transpose()

In [125]:
OnsiteUpdate.columns = OnsiteUpdate.iloc[0]

In [126]:
OnsiteUpdate = OnsiteUpdate[1:]


In [127]:
OnsiteUpdate=OnsiteUpdate[['Onboarded Onshore','Resigned','No Show','TotalOnshoreDemand']]

In [128]:
OnsiteUpdate

Attributes,Onboarded Onshore,Resigned,No Show,TotalOnshoreDemand
Counts,2,0,0,2


In [129]:
OnsiteUpdate.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\OnshoreMetrics_'+TribeName+'.csv',sep=',',index=None)

In [130]:
Resource = resourceResigned.append(resourceNoshow)

In [131]:
Resource

,Resource Name,Role,Onboarding Status


In [132]:
Resource.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\OnshoreRsgnNshw_'+TribeName+'.csv',sep=',',index=None)

Demand Status by Role

In [133]:
Demandtriberole=TribeDemandTracker
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Test Lead'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Environment Manager'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Operations Expert'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Agile Coach'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Solution Designer'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Developer'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Transition Manager'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Product Owner'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Lead Developer'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Journey Expert'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['SDET - Automation Tester'],['Tester'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['Chapter Lead'],['Other'])
Demandtriberole['Role'] = Demandtriberole['Role'].replace(['PMO Analyst'],['Other'])

In [134]:
Onboardingtriberole=DXCOnboarding
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Test Lead'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Environment Manager'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Operations Expert'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Agile Coach'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Solution Designer'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Developer'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Transition Manager'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Product Owner'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Lead Developer'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Journey Expert'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['SDET - Automation Tester'],['Tester'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Chapter Lead'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['PMO Analyst'],['Other'])
Onboardingtriberole['Role'] = Onboardingtriberole['Role'].replace(['Delivery Partner'],['Other'])

In [135]:
ro=Demandtriberole[Demandtriberole['Demand Status']=='Open']
ro=ro.groupby(['Role'])['ID'].count().reset_index(name='Roles Open')

In [136]:
Dcls=Demandtriberole[Demandtriberole['Demand Status']=='Closed']
Dcls=Dcls.groupby(['Role'])['ID'].count().reset_index(name='Demand Closed')

In [137]:
rorsgn=Onboardingtriberole[(Onboardingtriberole['Onboarding Status']=='Resigned')]
rorsgn=rorsgn.groupby(['Role'])['ID'].count().reset_index(name='Resigned')

In [138]:
roopnbckfil=Demandtriberole[Demandtriberole['Demand Status']=='Open Backfill']
roopnbckfil=roopnbckfil.groupby(['Role'])['ID'].count().reset_index(name='Roles Open Backfill')

In [139]:
robckfilcls=Demandtriberole[Demandtriberole['Demand Status']=='Backfill Closed']
robckfilcls=robckfilcls.groupby(['Role'])['ID'].count().reset_index(name='Roles Backfill Closed')

In [140]:
rowithdrw=Demandtriberole[Demandtriberole['Demand Status']=='Withdrawn']
rowithdrw=rowithdrw.groupby(['Role'])['ID'].count().reset_index(name='Withdrawn')

In [141]:
ronoshw=Onboardingtriberole[(Onboardingtriberole['Onboarding Status']=='No Show')]
ronoshw=ronoshw.groupby(['Role'])['ID'].count().reset_index(name='No Show')

In [142]:
DefaultRole = ['Engineer','Tester','Release Lead','Analyst','Tech Area Architect','Scrum Master','Other']

In [143]:
dfrole = pd.DataFrame(DefaultRole)

In [144]:
dfrole.columns=['Role']

In [145]:
dfrole

,Role
0,Engineer
1,Tester
2,Release Lead
3,Analyst
4,Tech Area Architect
5,Scrum Master
6,Other


In [146]:
drole = [ro,Dcls,rorsgn,roopnbckfil,robckfilcls,rowithdrw,ronoshw,dfrole]

In [147]:
DemandStatusRole=reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),drole)

In [148]:
DemandStatusRole = DemandStatusRole[['Role','Roles Open','Demand Closed','Resigned','Roles Open Backfill','Roles Backfill Closed','Withdrawn','No Show']]

In [149]:
DemandStatusRole = DemandStatusRole.fillna('0')

In [150]:
DemandStatusRole

,Role,Roles Open,Demand Closed,Resigned,Roles Open Backfill,Roles Backfill Closed,Withdrawn,No Show
0,Engineer,0,1,0,0,0,0,0
1,Other,0,2,0,0,0,0,0
2,Tester,0,0,0,0,0,0,0
3,Release Lead,0,0,0,0,0,0,0
4,Analyst,0,0,0,0,0,0,0
5,Tech Area Architect,0,0,0,0,0,0,0
6,Scrum Master,0,0,0,0,0,0,0


In [151]:
DemandStatusRole = DemandStatusRole.transpose()

In [152]:
DemandStatusRole

,0,1,2,3,4,5,6
Role,Engineer,Other,Tester,Release Lead,Analyst,Tech Area Architect,Scrum Master
Roles Open,0,0,0,0,0,0,0
Demand Closed,1,2,0,0,0,0,0
Resigned,0,0,0,0,0,0,0
Roles Open Backfill,0,0,0,0,0,0,0
Roles Backfill Closed,0,0,0,0,0,0,0
Withdrawn,0,0,0,0,0,0,0
No Show,0,0,0,0,0,0,0


In [153]:
DemandStatusRole.columns = DemandStatusRole.iloc[0]
DemandStatusRole = DemandStatusRole[1:]

In [154]:
DemandStatusRole

Role,Engineer,Other,Tester,Release Lead,Analyst,Tech Area Architect,Scrum Master
Roles Open,0,0,0,0,0,0,0
Demand Closed,1,2,0,0,0,0,0
Resigned,0,0,0,0,0,0,0
Roles Open Backfill,0,0,0,0,0,0,0
Roles Backfill Closed,0,0,0,0,0,0,0
Withdrawn,0,0,0,0,0,0,0
No Show,0,0,0,0,0,0,0


In [155]:
DemandStatusRole = DemandStatusRole[['Engineer','Tester','Release Lead','Analyst','Tech Area Architect','Scrum Master','Other']]

In [156]:
DemandStatusRole = DemandStatusRole.astype(int)

In [157]:
DemandStatusRole['Total']=DemandStatusRole['Engineer']+DemandStatusRole['Tester']+DemandStatusRole['Release Lead']+DemandStatusRole['Analyst']+DemandStatusRole['Tech Area Architect']+DemandStatusRole['Scrum Master']+DemandStatusRole['Other']

In [158]:
DemandStatusRole

Role,Engineer,Tester,Release Lead,Analyst,Tech Area Architect,Scrum Master,Other,Total
Roles Open,0,0,0,0,0,0,0,0
Demand Closed,1,0,0,0,0,0,2,3
Resigned,0,0,0,0,0,0,0,0
Roles Open Backfill,0,0,0,0,0,0,0,0
Roles Backfill Closed,0,0,0,0,0,0,0,0
Withdrawn,0,0,0,0,0,0,0,0
No Show,0,0,0,0,0,0,0,0


In [159]:
DemandStatusRole['Total']=DemandStatusRole['Total'].astype(int)

In [160]:
DemandStatusRole

Role,Engineer,Tester,Release Lead,Analyst,Tech Area Architect,Scrum Master,Other,Total
Roles Open,0,0,0,0,0,0,0,0
Demand Closed,1,0,0,0,0,0,2,3
Resigned,0,0,0,0,0,0,0,0
Roles Open Backfill,0,0,0,0,0,0,0,0
Roles Backfill Closed,0,0,0,0,0,0,0,0
Withdrawn,0,0,0,0,0,0,0,0
No Show,0,0,0,0,0,0,0,0


In [161]:
DemandStatusRole.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\'+TribeName+'\\Demand_Status_Role'+TribeName+'.csv',sep=',')

In [162]:
LapOnboarding = OnboardingTracker[OnboardingTracker['Device Assigned'] == True]

In [163]:
LapOnboarding = LapOnboarding[['Demand ID','Resource Name','Device Assigned','Device Type','Tribe','Squad']]

In [164]:
LapOnboarding['Device Type'].unique()

array(['Developer Laptop', 'Standard Laptop', 'Macbook Dev Edition', nan], dtype=object)

In [165]:
DevLap = LapOnboarding[LapOnboarding['Device Type'] == 'Developer Laptop']
DevLap = DevLap.fillna('')
DevLap.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\DevLaptop.csv',sep=',',index=None)

In [166]:
StdLap = LapOnboarding[LapOnboarding['Device Type'] == 'Standard Laptop']
StdLap = StdLap.fillna('')
StdLap.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\STDLaptop.csv',sep=',',index=None)

In [167]:
Mac = LapOnboarding[LapOnboarding['Device Type'] == 'Macbook Dev Edition']
Mac=Mac.fillna('')
Mac
Mac.to_csv('\\NotBackedUp\\SOE\\pyscript\\Tribe\\Macbook.csv',sep=',',index=None)